In [1]:
import pandas as pd
import pyspark
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn import preprocessing
import scipy

In [2]:
try:
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

In [3]:
avisos = pd.read_csv('Sources/fiuba_6_avisos_detalle_limpio.csv').drop('Unnamed: 0',axis=1)

In [4]:
stopwords = "el la los les las de del a ante con en para por y o u tu te ti le que al ha un han lo su una estas esto este es tras suya a acá ahí ajena ajenas ajeno ajenos al algo algún alguna algunas alguno algunos allá alli allí ambos ampleamos ante antes aquel aquella aquellas aquello aquellos aqui aquí arriba  asi atras aun aunque bajo bastante bien cabe cada casi cierta ciertas cierto ciertos como cómo con conmigo conseguimos conseguir consigo consigue consiguen consigues contigo contra cual cuales cualquier cualquiera cualquieras cuancuán cuando cuanta cuánta cuantas cuántas cuanto cuánto cuantos cuántos de dejar del demás demas demasiada demasiadas demasiado demasiados  dentro desde donde dos el él ella ellas  ello ellos empleais emplean emplear empleas empleo en encima entonces entre era eramos eran eras eres es esa esas ese eso esos esta estaba estado estais estamos estan estar estas este esto estos estoy etc fin fue fueron fui fuimos gueno ha hace haceis hacemos hacen hacer haces hacia"
stopwords = stopwords + "haago hasta incluso intenta intentais intentamos intentan intentar intentas intento ir jamás junto juntos la largo las lo los mas más me menos mi mía mia mias mientras mio mío mios mis misma mismas mismo mismos modo mucha muchas muchísima muchísimas muchísimo muchísimos mucho muchos muy nada ni ningun ninguna ningunas ninguno ningunos no nos nosotras nosotros nuestra nuestras nuestro nuestros nunca os otra otras otro otros para parecer pero poca pocas poco pocos podeis podemos poder podria podriais podriamos podrian podrias por por qué porque primero primero desde puede pueden puedo pues que qué querer quien quién quienes quienes quiera quienquiera quiza quizas sabe sabeis sabemos saben saber sabes se segun ser si sí siempre siendo sin sín sino so sobre sois solamente solo somos soy sr sra sres esta su sus suya suyas suyo suyos tal tales también tambien tampoco tan tanta tantas tanto tantos te teneis tenemos tener tengo ti tiempo tiene tienen toda todas todo todos tomar trabaja"
stopwords = stopwords +  "trabajais trabajamos trabajan trabajar trabajas trabajo tras tú tu tus tuya tuyo tuyos ultimo un una unas uno unos usa usais usamos usan usar usas uso usted ustedes va vais valor vamos van varias varios vaya verdad verdadera vosotras vosotros voy vuestra vuestras vuestro vuestros y ya yo como cómo hacer se tengo"
stopwords = stopwords.split(' ')

In [5]:
titulos = avisos['titulo']
print(titulos.shape)
titulos = titulos.fillna('')
vectorizer = TfidfVectorizer(stop_words=stopwords)
titulos = vectorizer.fit_transform(titulos)
print(titulos.toarray().shape)

(25288,)
(25288, 6995)


In [6]:
titulos = titulos.toarray()
df = pd.DataFrame.from_records(titulos)
df['nombre_area'] = avisos['nombre_area']

In [7]:
le = preprocessing.LabelEncoder()
df['nombre_area'] = df['nombre_area'].fillna('na')
le.fit(df['nombre_area'])
df['nombre_area'] = le.transform(df['nombre_area'])
print(df.shape)

(25288, 6996)


In [8]:
df = scipy.sparse.csr_matrix(df.values)

In [9]:
df.shape
kmeanModel = KMeans(n_clusters=228, init='k-means++', max_iter=100, n_init=30, n_jobs=-1)
kmeanModel = kmeanModel.fit(df)
clusters = kmeanModel.predict(df)

In [10]:
print(avisos.shape)
print(clusters.shape)

(25288, 7)
(25288,)


In [12]:
cluster = pd.DataFrame()
cluster['idaviso'] = avisos['idaviso']
cluster['cluster'] = clusters

In [14]:
print(cluster.shape)
cluster.to_csv('Sources/idaviso-cluster.csv')

(25288, 2)
